## Import

In [1]:
!pip install accelerate

In [2]:
!pip install optuna

In [3]:
!pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126

Looking in indexes: https://download.pytorch.org/whl/cu126


In [4]:
!pip install xformers

## Drive, HuggingFace, Accelerate

In [11]:
%cd ./diffusers

[Errno 2] No such file or directory: './diffusers'
/workspace/PFE/models/diffusers/examples/text_to_image


In [3]:
%ls

CITATION.cff        MANIFEST.in    _typos.toml  examples/       src/
CODE_OF_CONDUCT.md  Makefile       benchmarks/  pyproject.toml  tests/
CONTRIBUTING.md     PHILOSOPHY.md  docker/      scripts/        utils/
LICENSE             README.md      docs/        setup.py


In [4]:
%cd examples/text_to_image/

/workspace/PFE/models/diffusers/examples/text_to_image


In [7]:
from huggingface_hub import notebook_login
notebook_login()

In [8]:
%ls

README.md
README_sdxl.md
optuna-sd-model_bs8_lr0.0001_schedcosine_acc4_0/
optuna-sd-model_bs8_lr5e-05_schedlinear_acc4_0/
plot_contour.html
plot_optimization_history.html
plot_parallel_coordinates.html
plot_param_importance.html
plot_slice.html
requirements.txt
requirements_flax.txt
requirements_sdxl.txt
test_text_to_image.py
test_text_to_image_lora.py
train_text_to_image.py
train_text_to_image_flax.py
train_text_to_image_lora.py
train_text_to_image_lora_sdxl.py
train_text_to_image_sdxl.py


In [9]:
!accelerate config default

Configuration already exists at /root/.cache/huggingface/accelerate/default_config.yaml, will not override. Run `accelerate config` manually or pass a different `save_location`.


In [7]:
# Remove current diffusers
!pip uninstall -y diffusers

# Install latest diffusers from GitHub (source install)
!pip install git+https://github.com/huggingface/diffusers.git


Found existing installation: diffusers 0.34.0.dev0
Uninstalling diffusers-0.34.0.dev0:
  Successfully uninstalled diffusers-0.34.0.dev0
  Cloning https://github.com/huggingface/diffusers.git to /tmp/pip-req-build-e3yyr69j
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers.git /tmp/pip-req-build-e3yyr69j
  Resolved https://github.com/huggingface/diffusers.git to commit 9836f0e000cfd826a7a5099002253ed2becc13e0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.34.0.dev0-py3-none-any.whl size=3687437 sha256=909400beb40b6b25ffd51f1bcc4393a29729e39f119a31a5d4715192fd0189f7
  Stored in directory: /tmp/pip-ephem-wheel-cache-c_zw_02u/wheels/23/0f/7d/f97813d265ed0e599a78d83afd4e1925740896ca79b46cccfd
Successfully built diffusers


In [8]:
!pip install -U datasets

Les meilleurs modeles:

- SD-1.5 \\
- SD-2.1 \\
- Realistic_Vision_V6.0 \\
- SDXL
- FLUX


### REALVISION1.5

In [ ]:
!accelerate launch train_text_to_image_lora.py \
  --pretrained_model_name_or_path="SG161222/Realistic_Vision_V5.1_noVAE" \
  --train_data_dir="/content/drive/MyDrive/PFE/mini/mini/images" --caption_column="text" \
  --dataloader_num_workers=8 \
  --resolution=512 \
  --train_batch_size=1 \
  --max_train_steps=200 \
  --max_grad_norm=1 \
  --gradient_accumulation_steps=4 \
  --num_train_epochs=2 --checkpointing_steps=500 \
  --learning_rate=1e-04 --lr_scheduler="cosine" --lr_warmup_steps=0 \
  --mixed_precision="fp16" \
  --seed=1337 \
  --output_dir="sd-tryon-realistic_vision-lora" \
  --validation_prompt="a slim white woman in thirties." \
  --push_to_hub

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2025-05-05 08:28:18.597767: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746433698.627617   12012 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746433698.637566   12012 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
05/05/2025 08:28:52 - INFO - __main__ - Distributed en

In [ ]:
from diffusers import DiffusionPipeline
import torch

model_path = "/content/diffusers/examples/text_to_image/sd-tryon-realistic_vision-lora"
pipe = DiffusionPipeline.from_pretrained("SG161222/Realistic_Vision_V5.1_noVAE", torch_dtype=torch.float16)
pipe.to("cuda")
pipe.load_lora_weights(model_path)

prompt = "A white average woman wearing white clothes in twenties. White background. full body image."
image = pipe(prompt, num_inference_steps=30, guidance_scale=7.5).images[0]
image.save("/content/img.png")

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

No LoRA keys associated to CLIPTextModel found with the prefix='text_encoder'. This is safe to ignore if LoRA state dict didn't originally have any CLIPTextModel related params. You can also try specifying `prefix=None` to resolve the warning. Otherwise, open an issue if you think it's unexpected: https://github.com/huggingface/diffusers/issues/new


  0%|          | 0/30 [00:00<?, ?it/s]

## SDXL

In [12]:
import optuna
import subprocess
import joblib
import optuna.visualization as vis
from datetime import datetime
import torch
import sys


# ========== OBJECTIVE FUNCTION ==========
def objective(trial):
    # Hyperparamètres à optimiser
    learning_rate = trial.suggest_categorical("learning_rate", [1e-5, 5e-5, 1e-4, 1e-6])
    batch_size = trial.suggest_categorical("train_batch_size", [8, 16])
    lr_scheduler = trial.suggest_categorical("lr_scheduler", ["cosine", "linear"])
    gradient_accumulation = trial.suggest_categorical("gradient_accumulation_steps", [4, 8])
    snr_gamma = trial.suggest_categorical('snr_gamma', [None, 5.0])
    rank = trial.suggest_categorical('rank', [4, 8, 16])
    # Nom unique pour chaque essai
    output_dir = f"optuna-sd-model_bs{batch_size}_lr{learning_rate}_sched{lr_scheduler}_acc{gradient_accumulation}_{trial.number}"

    # Commande d'entraînement
    command = [
        "accelerate", "launch", "train_text_to_image_lora_sdxl.py",
        "--pretrained_model_name_or_path=stabilityai/stable-diffusion-xl-base-1.0",
        "--pretrained_vae_model_name_or_path=madebyollin/sdxl-vae-fp16-fix",
        "--train_data_dir=/workspace/PFE/VITON-HD-512/train_images",
        "--caption_column=text",
        "--dataloader_num_workers=8",
        "--resolution=512",
        f"--train_batch_size={batch_size}",
        #f"--max_train_steps={4052 if batch_size == 8 else 2026}",
        "--max_grad_norm=1",
        f"--gradient_accumulation_steps={gradient_accumulation}",
        "--num_train_epochs=3",
        "--checkpointing_steps=2026",
        f"--learning_rate={learning_rate}",
        f"--lr_scheduler={lr_scheduler}",
        "--lr_warmup_steps=0",
        "--mixed_precision=fp16",
        "--seed=1337",
        "--allow_tf32",
        "--use_8bit_adam",
        "--enable_xformers_memory_efficient_attention",
        f"--rank={rank}",
        f"--output_dir={output_dir}"
        #"--validation_prompt=A portrait of a pregnant black woman in her thirties, plain background.",
        #"--num_validation_images=0"    
    ]

    if snr_gamma is not None:
        f"--snr_gamma={snr_gamma}",


    try:
        torch.cuda.empty_cache()
        print(f"🔁 Lancement de l'entraînement pour les params : {trial.params}")
        result = subprocess.run(
            command, 
             stdout=subprocess.PIPE,
            #stderr=subprocess.STDOUT,
            text=True,
            check=True)
    except Exception as e:
        print("Erreur lors du nettoyage GPU :", e)

  
    if 'total_loss' in result.stdout:
        for line in result.stdout.splitlines():
            if "total_loss" in line:
                try:
                    loss = float(line.split("total_loss")[1].split()[1])
                    print(f"[Trial {trial.number}] optuna_metric: {loss}")
                    return loss
                except Exception as e:
                    print(f"[Trial {trial.number}] Erreur extraction train_loss:", e)

    return float("inf")
            
# ========== LANCER L’OPTIMISATION ==========

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
study_name = f"sdxl_optuna_study_{timestamp}"
study = optuna.create_study(direction="minimize", study_name=study_name)
study.optimize(objective, n_trials=20)
# Sauvegarde du modèle d’étude
joblib.dump(study, f"{study_name}.pkl")
print("\n=============================")
print("💡 Best trial:")
print(study.best_trial)
print("=============================")

[I 2025-05-17 15:51:02,431] A new study created in memory with name: sdxl_optuna_study_20250517_155102


🔁 Lancement de l'entraînement pour les params : {'learning_rate': 5e-05, 'train_batch_size': 16, 'lr_scheduler': 'cosine', 'gradient_accumulation_steps': 4, 'snr_gamma': 5.0, 'rank': 4}


/venv/main/lib/python3.12/site-packages/accelerate/accelerator.py:498: UserWarning: `log_with=tensorboard` was passed but no supported trackers are currently installed.
  warnings.warn(f"`log_with={log_with}` was passed but no supported trackers are currently installed.")
05/17/2025 15:51:09 - INFO - __main__ - Distributed environment: DistributedType.NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: fp16

You are using a model of type clip_text_model to instantiate a model of type . This is not supported for all configurations of models and can yield errors.
You are using a model of type clip_text_model to instantiate a model of type . This is not supported for all configurations of models and can yield errors.
{'rescale_betas_zero_snr', 'variance_type', 'dynamic_thresholding_ratio', 'clip_sample_range', 'thresholding'} was not found in config. Values will be initialized to default values.
{'shift_factor', 'latents_mean', 'latents_std', 'm

[Trial 0] optuna_metric: 0.09111547058514195
🔁 Lancement de l'entraînement pour les params : {'learning_rate': 5e-05, 'train_batch_size': 16, 'lr_scheduler': 'linear', 'gradient_accumulation_steps': 4, 'snr_gamma': None, 'rank': 8}


/venv/main/lib/python3.12/site-packages/accelerate/accelerator.py:498: UserWarning: `log_with=tensorboard` was passed but no supported trackers are currently installed.
  warnings.warn(f"`log_with={log_with}` was passed but no supported trackers are currently installed.")
05/17/2025 16:31:43 - INFO - __main__ - Distributed environment: DistributedType.NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: fp16

You are using a model of type clip_text_model to instantiate a model of type . This is not supported for all configurations of models and can yield errors.
You are using a model of type clip_text_model to instantiate a model of type . This is not supported for all configurations of models and can yield errors.
{'dynamic_thresholding_ratio', 'variance_type', 'rescale_betas_zero_snr', 'thresholding', 'clip_sample_range'} was not found in config. Values will be initialized to default values.
{'shift_factor', 'mid_block_add_attention', 'use_p

...

[Trial 17] optuna_metric: 0.09035166661305993
🔁 Lancement de l'entraînement pour les params : {'learning_rate': 0.0001, 'train_batch_size': 16, 'lr_scheduler': 'cosine', 'gradient_accumulation_steps': 8, 'snr_gamma': 5.0, 'rank': 4}


/venv/main/lib/python3.12/site-packages/accelerate/accelerator.py:498: UserWarning: `log_with=tensorboard` was passed but no supported trackers are currently installed.
  warnings.warn(f"`log_with={log_with}` was passed but no supported trackers are currently installed.")
05/18/2025 04:28:52 - INFO - __main__ - Distributed environment: DistributedType.NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: fp16

You are using a model of type clip_text_model to instantiate a model of type . This is not supported for all configurations of models and can yield errors.
You are using a model of type clip_text_model to instantiate a model of type . This is not supported for all configurations of models and can yield errors.
{'rescale_betas_zero_snr', 'clip_sample_range', 'variance_type', 'thresholding', 'dynamic_thresholding_ratio'} was not found in config. Values will be initialized to default values.
{'use_quant_conv', 'shift_factor', 'use_post_quant

[Trial 18] optuna_metric: 0.090338858346513
🔁 Lancement de l'entraînement pour les params : {'learning_rate': 1e-05, 'train_batch_size': 16, 'lr_scheduler': 'cosine', 'gradient_accumulation_steps': 8, 'snr_gamma': 5.0, 'rank': 4}


/venv/main/lib/python3.12/site-packages/accelerate/accelerator.py:498: UserWarning: `log_with=tensorboard` was passed but no supported trackers are currently installed.
  warnings.warn(f"`log_with={log_with}` was passed but no supported trackers are currently installed.")
05/18/2025 05:09:05 - INFO - __main__ - Distributed environment: DistributedType.NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: fp16

You are using a model of type clip_text_model to instantiate a model of type . This is not supported for all configurations of models and can yield errors.
You are using a model of type clip_text_model to instantiate a model of type . This is not supported for all configurations of models and can yield errors.
{'variance_type', 'clip_sample_range', 'rescale_betas_zero_snr', 'dynamic_thresholding_ratio', 'thresholding'} was not found in config. Values will be initialized to default values.
{'shift_factor', 'latents_std', 'latents_mean', 'u

[Trial 19] optuna_metric: 0.09253397656902962

💡 Best trial:
FrozenTrial(number=18, state=1, values=[0.090338858346513], datetime_start=datetime.datetime(2025, 5, 18, 4, 28, 45, 71078), datetime_complete=datetime.datetime(2025, 5, 18, 5, 8, 57, 980954), params={'learning_rate': 0.0001, 'train_batch_size': 16, 'lr_scheduler': 'cosine', 'gradient_accumulation_steps': 8, 'snr_gamma': 5.0, 'rank': 4}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'learning_rate': CategoricalDistribution(choices=(1e-05, 5e-05, 0.0001, 1e-06)), 'train_batch_size': CategoricalDistribution(choices=(8, 16)), 'lr_scheduler': CategoricalDistribution(choices=('cosine', 'linear')), 'gradient_accumulation_steps': CategoricalDistribution(choices=(4, 8)), 'snr_gamma': CategoricalDistribution(choices=(None, 5.0)), 'rank': CategoricalDistribution(choices=(4, 8, 16))}, trial_id=18, value=None)


In [13]:
# ========== VISUALISATION ==========

import optuna.visualization as vis
print("📊 Génération des graphiques Optuna...")
vis.plot_optimization_history(study).write_html("plot_optimization_history.html", include_plotlyjs="cdn")
vis.plot_param_importances(study).write_html("plot_param_importance.html", include_plotlyjs="cdn")
vis.plot_parallel_coordinate(study).write_html("plot_parallel_coordinates.html", include_plotlyjs="cdn")
vis.plot_slice(study).write_html("plot_slice.html", include_plotlyjs="cdn")
vis.plot_contour(study, params=["learning_rate", "gradient_accumulation_steps"]).write_html("plot_contour.html", include_plotlyjs="cdn")
print("✅ Plots sauvegardés en fichiers HTML.")

📊 Génération des graphiques Optuna...
✅ Plots sauvegardés en fichiers HTML.


In [16]:
from diffusers import DiffusionPipeline
import torch

model_path = "/workspace/PFE/models/diffusers/examples/text_to_image/optuna-sd-model_bs16_lr0.0001_schedcosine_acc8_18/"
pipe = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16)
pipe.to("cuda")
pipe.load_lora_weights(model_path)

prompt = "A slim white woman in her twenties, white background. full body."
image = pipe(prompt, num_inference_steps=30, guidance_scale=7.5).images[0]
image.save("/workspace/PFE/models/img.png")

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

No LoRA keys associated to CLIPTextModel found with the prefix='text_encoder'. This is safe to ignore if LoRA state dict didn't originally have any CLIPTextModel related params. You can also try specifying `prefix=None` to resolve the warning. Otherwise, open an issue if you think it's unexpected: https://github.com/huggingface/diffusers/issues/new
No LoRA keys associated to CLIPTextModelWithProjection found with the prefix='text_encoder_2'. This is safe to ignore if LoRA state dict didn't originally have any CLIPTextModelWithProjection related params. You can also try specifying `prefix=None` to resolve the warning. Otherwise, open an issue if you think it's unexpected: https://github.com/huggingface/diffusers/issues/new


  0%|          | 0/30 [00:00<?, ?it/s]

## SD-2.1

In [ ]:
!pip install bitsandbytes

In [ ]:
import optuna
import subprocess
import joblib
import optuna.visualization as vis
from datetime import datetime
import sys
# ========== OBJECTIVE FUNCTION ==========
def objective(trial):
    # Hyperparamètres à optimiser
    learning_rate = trial.suggest_categorical("learning_rate", [1e-5, 5e-5, 1e-4, 1e-6])
    batch_size = trial.suggest_categorical("train_batch_size", [8])
    lr_scheduler = trial.suggest_categorical("lr_scheduler", ["cosine", "linear"])
    gradient_accumulation = trial.suggest_categorical("gradient_accumulation_steps", [4, 8])

    # Nom unique pour chaque essai
    output_dir = f"optuna-sd-model_bs{batch_size}_lr{learning_rate}_sched{lr_scheduler}_acc{gradient_accumulation}_{trial.number}"

    # Commande d'entraînement
    command = [
        "accelerate", "launch", "train_text_to_image_lora.py",
        "--pretrained_model_name_or_path=stabilityai/stable-diffusion-2-1",
        "--train_data_dir=/content/drive/MyDrive/PFE/main/PFE/dataset/images",
        "--caption_column=prompt",
        "--dataloader_num_workers=8",
        "--resolution=512",
        f"--train_batch_size={batch_size}",
        "--max_train_steps=10",
        "--max_grad_norm=1",
        f"--gradient_accumulation_steps={gradient_accumulation}",
        "--num_train_epochs=2",
        "--checkpointing_steps=150",
        f"--learning_rate={learning_rate}",
        f"--lr_scheduler={lr_scheduler}",
        "--lr_warmup_steps=0",
        "--mixed_precision=fp16",
        "--seed=1337",
        '--allow_tf32',
        '--use_8bit_adam',
        f"--output_dir={output_dir}",
        "--validation_prompt=A portrait of a slim white woman in her twenties, plain background."

    ]

    print(f"\n🚀 [Trial {trial.number}] Commande:\n{' '.join(command)}\n")

    # Exécution avec affichage direct
    #---------------------------------------

    import shlex
    process = subprocess.Popen(
    command,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1
    )

    log_lines = []
    for line in process.stdout:
      print(line, end="")  # Affiche chaque ligne en temps réel
      log_lines.append(line)

    process.wait()
    output_text = "".join(log_lines)

    #result = subprocess.run(command, capture_output=True, text=True)
    #print(result.stderr)
    #print('-'*30)
    #print(result.stdout)
    # Extraction de la perte depuis stdout
    if "train_loss" in output_text:
        for line in output_text.splitlines():
            if "train_loss" in line:
                try:
                    loss = float(line.split("train_loss")[1].split()[1])
                    print(f"[Trial {trial.number}] train_loss: {loss}")
                    print(loss)
                    return loss
                except Exception as e:
                    print(f"[Trial {trial.number}] Erreur extraction train_loss:", e)

    return float("inf")  # Mauvais essai si échec


# ========== LANCER L’OPTIMISATION ==========

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
study_name = f"sd_optuna_study_{timestamp}"
study = optuna.create_study(direction="minimize", study_name=study_name)
study.optimize(objective, n_trials=5)
# Sauvegarde du modèle d’étude
joblib.dump(study, f"{study_name}.pkl")
print("\n=============================")
print("💡 Best trial:")
print(study.best_trial)



In [ ]:
import math
print("📊 Génération des graphiques Optuna...")

# Ne garder que les essais réussis
completed_trials = [t for t in study.trials if t.value is not None and math.isfinite(t.value)]
print(completed_trials)
if len(completed_trials) < 2:
    print("⚠️ Pas assez de résultats valides pour les visualisations avancées.")
    vis.plot_optimization_history(study).write_html("plot_optimization_history.html", include_plotlyjs="cdn")
else:
    vis.plot_optimization_history(study).write_html("plot_optimization_history.html", include_plotlyjs="cdn")
    vis.plot_param_importances(study).write_html("plot_param_importance.html", include_plotlyjs="cdn")
    vis.plot_parallel_coordinate(study).write_html("plot_parallel_coordinates.html", include_plotlyjs="cdn")
    vis.plot_slice(study).write_html("plot_slice.html", include_plotlyjs="cdn")
    vis.plot_contour(study, params=["learning_rate", "gradient_accumulation_steps"]).write_html("plot_contour.html", include_plotlyjs="cdn")

print("✅ Plots sauvegardés en fichiers HTML.")



In [29]:
from diffusers import DiffusionPipeline
import torch

model_path = "/content/drive/MyDrive/PFE/main/PFE/models/diffusers/examples/text_to_image/optuna-sd-model_bs8_lr1e-05_schedlinear_acc8_0"
pipe = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-2-1", torch_dtype=torch.float16)
pipe.to("cuda")
pipe.load_lora_weights(model_path)

prompt = "A portrait of a pregnant white woman in her twenties, plain background."
image = pipe(prompt, num_inference_steps=30, guidance_scale=7.5).images[0]
image.save("/content/img.png")

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

No LoRA keys associated to CLIPTextModel found with the prefix='text_encoder'. This is safe to ignore if LoRA state dict didn't originally have any CLIPTextModel related params. You can also try specifying `prefix=None` to resolve the warning. Otherwise, open an issue if you think it's unexpected: https://github.com/huggingface/diffusers/issues/new


  0%|          | 0/30 [00:00<?, ?it/s]